In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
from datetime import timedelta
from scipy import spatial
from scipy.sparse.linalg import svds
from scipy import sparse
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

rec_week_num = 43
rec_week_year = 2018
week_delta = 8

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
%pylab inline 

['material_cis_merge.csv', 'lesson_plans.csv', 'mat_actions_clean.csv']
Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
%%time
df = pd.read_csv('../input/mat_actions_clean.csv', sep = ',')
#df.action_start = df.action_start.apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))

<string>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 14.5 s, sys: 1.84 s, total: 16.3 s
Wall time: 16.4 s


In [3]:
df.head(2)

,action_id,material_type,material_id,user_id,average_rating,action_type,action_info,action_start,action_user_id,real_type,subject,ed_level
0,0,AtomicObject,1182,425445.0,4.6,оценка,5,2015-07-17 13:22:47.446491,425445.0,текст,Физкультура,ООО
1,1,AtomicObject,1308,425445.0,4.0,оценка,5,2015-07-17 13:41:49.001476,425445.0,текст,Физкультура,ООО


In [4]:
from datetime import datetime

df.action_start = df.action_start.apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))

In [5]:
df.head(2)

,action_id,material_type,material_id,user_id,average_rating,action_type,action_info,action_start,action_user_id,real_type,subject,ed_level
0,0,AtomicObject,1182,425445.0,4.6,оценка,5,2015-07-17,425445.0,текст,Физкультура,ООО
1,1,AtomicObject,1308,425445.0,4.0,оценка,5,2015-07-17,425445.0,текст,Физкультура,ООО


In [6]:
%%time
mc = pd.read_csv('../input/material_cis_merge.csv', sep = ',')
mc = mc.rename(columns = {'education_level': 'ed_level'}, index = str)

CPU times: user 3.06 s, sys: 192 ms, total: 3.25 s
Wall time: 3.27 s


In [7]:
mc.head(2)

,material_type,material_id,name,code,subject,ed_level,user_id,created_at,real_type
0,AtomicObject,38,История и современное развитие физической куль...,1,Физическая культура,ООО,423768.0,2015-04-24 12:00:51.054082,изображение
1,AtomicObject,39,История и современное развитие физической куль...,1,Физическая культура,ООО,423768.0,2015-04-24 12:00:54.291523,изображение


In [8]:
lp = pd.read_csv('../input/lesson_plans.csv', sep = ',')

In [9]:
lp.head(2)

,lesson_id,teacher_id,date,subject,ed_level,code,cis
0,74934616,4145840,2018-09-04,Изобразительное искусство,НОО,1.2,Рисунок
1,74934661,4145840,2018-09-11,Изобразительное искусство,НОО,1.1,Восприятие произведений искусства


In [10]:
lp.shape

(6199645, 7)

In [11]:
lp.teacher_id.unique().shape

(19202,)

In [12]:
df[df.action_user_id.isin(lp.teacher_id.unique())].action_user_id.unique().shape

(506,)

In [13]:
df.action_user_id.unique().shape

(175503,)

In [14]:
lp.date = lp.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [15]:
lp.head(2)

,lesson_id,teacher_id,date,subject,ed_level,code,cis
0,74934616,4145840,2018-09-04,Изобразительное искусство,НОО,1.2,Рисунок
1,74934661,4145840,2018-09-11,Изобразительное искусство,НОО,1.1,Восприятие произведений искусства


#### Добавляем столбцы со значениями номера недели и года

In [16]:
lp['week_num'] = lp.date.apply(lambda x: x.isocalendar()[1])
lp['year_num'] = lp.date.apply(lambda x: x.isocalendar()[0])

In [17]:
lp.head(2)

,lesson_id,teacher_id,date,subject,ed_level,code,cis,week_num,year_num
0,74934616,4145840,2018-09-04,Изобразительное искусство,НОО,1.2,Рисунок,36,2018
1,74934661,4145840,2018-09-11,Изобразительное искусство,НОО,1.1,Восприятие произведений искусства,37,2018


In [18]:
rec_week_num = 43

#### Создаём таблицу, в которой будут значения, соответствующие запрашиваемой недели

In [19]:
tmp = lp.loc[lp.week_num == rec_week_num, ['teacher_id', 'code', 'cis', 'week_num', 'subject', 'ed_level']
            ].groupby(['teacher_id', 'code', 'cis', 'subject', 'ed_level']).count().reset_index().iloc[:, :5]
tmp = tmp[tmp.code.apply(lambda x: len(str(x)) > 2)]
tmp.head(2)

,teacher_id,code,cis,subject,ed_level
0,49,4.1.7,Лингвистические словари. Работа со словарной с...,Русский язык,ООО
1,49,4.1.8,Выдающиеся отечественные лингвисты.,Русский язык,ООО


#### Создаём таблицу со значениями оценок и количестве голосов оценивания

In [102]:
%%time
# Создаём таблицу со значениями оценок и количества голосов оценивания

marks = df.loc[df.action_type == 'оценка', ['material_id', 'material_type', 'action_id', 'action_info']]
marks.action_info = marks.action_info.apply(float)
marks = marks.groupby(['material_id', 'material_type', 'action_id']).max().reset_index()
marks = pd.merge(marks.groupby(['material_id', 'material_type']).action_info.mean().reset_index(),
                 marks.groupby(['material_id', 'material_type']).action_id.count().reset_index(),
                 how = 'inner', on = ['material_id', 'material_type']
                ).rename(columns = {'action_info': 'avg_rating', 'action_id': 'marks_qty'}, index = str)

CPU times: user 3.45 s, sys: 8 ms, total: 3.46 s
Wall time: 3.45 s


In [21]:
marks.head(2)

,material_id,material_type,avg_rating,marks_qty
0,1,Book,5.0,3
1,2,Book,5.0,3


Смотрим на среднюю оценку

In [22]:
teacher_id = 49
rec = pd.merge(pd.merge(tmp.loc[tmp.teacher_id == teacher_id, ['teacher_id', 'code', 'ed_level', 'subject']], mc, 
                        how = 'inner', on = ['code', 'ed_level', 'subject']
                       ).groupby(['material_type', 'material_id']).user_id.count().reset_index().iloc[:, :2], 
               marks, how = 'inner', on = ['material_type', 'material_id'])
rec.head(5)

,material_type,material_id,avg_rating,marks_qty
0,AtomicObject,10059,3.000000,2
1,AtomicObject,47352,4.000000,1
2,AtomicObject,108058,4.000000,1
3,AtomicObject,369173,3.666667,3
4,AtomicObject,377286,5.000000,3


#### можно посмотреть что будет если поменять принцип ранжирования, например на тот что снизу
#rec.sort_values(by = ['marks_qty', 'avg_rating'], ascending = False).head(10)

#### Ранжируем их по принципу самые высокие и высоко оцениваемые материалы

In [23]:
rec.sort_values(by = ['avg_rating', 'marks_qty'], ascending = False).head(10)

,material_type,material_id,avg_rating,marks_qty
83,AtomicObject,1131825,5.0,46
78,AtomicObject,1131804,5.0,45
80,AtomicObject,1131810,5.0,43
89,AtomicObject,1131851,5.0,43
79,AtomicObject,1131807,5.0,42
85,AtomicObject,1131830,5.0,42
20,AtomicObject,430836,5.0,41
81,AtomicObject,1131811,5.0,41
84,AtomicObject,1131828,5.0,41
88,AtomicObject,1131848,5.0,41


### Создаём функцию, для подбора рекомендаций материала

In [24]:
def make_recom_by_les_plan(rec_week_num, rec_week_year, teacher_id):
    tmp = lp.loc[(lp.week_num == rec_week_num) & (lp.year_num == rec_week_year), 
                 ['teacher_id', 'code', 'cis', 'week_num', 'subject', 'ed_level']
                ].groupby(['teacher_id', 'code', 'cis', 'subject', 'ed_level']).count().reset_index().iloc[:, :5]
    tmp = tmp[tmp.code.apply(lambda x: len(str(x)) > 2)]
    rec = pd.merge(pd.merge(tmp.loc[tmp.teacher_id == teacher_id, ['teacher_id', 'code', 'ed_level', 'subject']], 
                            mc, how = 'inner', on = ['code', 'ed_level', 'subject']
                           ).groupby(['material_type', 'material_id']).user_id.count().reset_index().iloc[:, :2], 
                   marks, how = 'inner', on = ['material_type', 'material_id'])
    return rec.sort_values(by = ['avg_rating', 'marks_qty'], ascending = False).head(10)

In [25]:
%%time
make_recom_by_les_plan(43, 2018, 49)

CPU times: user 716 ms, sys: 108 ms, total: 824 ms
Wall time: 823 ms


,material_type,material_id,avg_rating,marks_qty
83,AtomicObject,1131825,5.0,46
78,AtomicObject,1131804,5.0,45
80,AtomicObject,1131810,5.0,43
89,AtomicObject,1131851,5.0,43
79,AtomicObject,1131807,5.0,42
85,AtomicObject,1131830,5.0,42
20,AtomicObject,430836,5.0,41
81,AtomicObject,1131811,5.0,41
84,AtomicObject,1131828,5.0,41
88,AtomicObject,1131848,5.0,41


#### Разбираемся с учителями, не имеющими поурочных планов

In [26]:
df[~df.action_user_id.isin(lp.teacher_id.unique())].action_user_id.unique()

array([  425445.,   425471.,   442589., ..., 12207477.,  5455196.,
        4725729.])

In [27]:
lp[lp.teacher_id == 425445]

,lesson_id,teacher_id,date,subject,ed_level,code,cis,week_num,year_num


In [28]:
df.head(2)

,action_id,material_type,material_id,user_id,average_rating,action_type,action_info,action_start,action_user_id,real_type,subject,ed_level
0,0,AtomicObject,1182,425445.0,4.6,оценка,5,2015-07-17,425445.0,текст,Физкультура,ООО
1,1,AtomicObject,1308,425445.0,4.0,оценка,5,2015-07-17,425445.0,текст,Физкультура,ООО


In [40]:
week_delta = 8

#### Смотрим на действия пользователей за последние n = 8 недель

In [ ]:
tmp = df[df.action_start.apply(lambda x: x.isocalendar()[1] < rec_week_num
                              ) & (df.action_start.apply(lambda x: x.isocalendar()[1] >= rec_week_num - week_delta))]

In [ ]:
tmp.head(2)

In [ ]:
tmp.shape

In [ ]:
tmp = tmp.groupby(['material_type', 'material_id', 'action_user_id']).action_id.count().reset_index().iloc[:, :3]

In [ ]:
tmp.shape

In [ ]:
tmp.head(5)

#### Склеиваем с информацией о материалах

In [ ]:
%%time
tmp = pd.merge(tmp, mc.iloc[:, :6], how = 'inner', on = ['material_type', 'material_id'])

In [ ]:
tmp.shape

In [ ]:
tmp.head(5)

In [ ]:
tmp.drop('name', axis = 1 , inplace = True)

In [ ]:
tmp.head(2)

#### Групируем столбцы, которые дают информацию о действиях и удаляем однозначные значения кода

In [ ]:
tmp = tmp.groupby(['action_user_id', 'code', 'subject', 'ed_level']).material_id.count().reset_index().iloc[:, :4]

In [ ]:
tmp = tmp[tmp.code.apply(lambda x: len(str(x)) > 2)]

In [ ]:
tmp.shape

In [ ]:
tmp.head(2)

In [ ]:
tmp.action_user_id.unique().shape

In [ ]:
t = tmp.groupby(['code', 'subject', 'ed_level']).count().reset_index()
tmp = pd.merge(tmp, t.loc[t.action_user_id > 1, ['code', 'subject', 'ed_level']], 
               how = 'inner', on = ['code', 'subject', 'ed_level'])

In [ ]:
tmp[(tmp.code == '1.0.4.3')]

In [ ]:
tmp.action_user_id.unique().shape

In [ ]:
tmp.groupby(['code', 'subject', 'ed_level']).count().shape

In [ ]:
%%time
tmp['theme'] = list(map(lambda x, y, z: x + '_' + y + '_' + z, tmp.code, tmp.subject, tmp.ed_level))

In [ ]:
tmp.drop(['code', 'subject', 'ed_level'], axis = 1, inplace = True)

In [ ]:
tmp.head(2)

Сделать список релевантных тем для определенного юзера

In [ ]:
teacher_id = 229275

In [ ]:
tmp[tmp.action_user_id == teacher_id].theme.values

In [ ]:
mc.head(2)

In [29]:
def make_recom_by_actions(rec_week_num, rec_week_year):
    week_delta = 8
    # отобрали все действия пользователей за предшествующие week_delta недель
    tmp = df[df.action_start.apply(lambda x: x.isocalendar()[1] < rec_week_num) & (
        df.action_start.apply(lambda x: x.isocalendar()[1] >= rec_week_num - week_delta)) & (
        df.action_start.apply(lambda x: x.isocalendar()[0] == rec_week_year))]
    # нашли все просматриваемые пользователем материалы
    tmp = tmp.groupby(['material_type', 'material_id', 'action_user_id']
                     ).action_id.count().reset_index().iloc[:, :3]
    # связали их с темами
    tmp = pd.merge(tmp, mc.iloc[:, :6], how = 'inner', on = ['material_type', 'material_id'])
    tmp.drop('name', axis = 1 , inplace = True)
    # получили датафрейм всех пользователей и тем
    tmp = tmp.groupby(['action_user_id', 'code', 'subject', 'ed_level']
                     ).material_id.count().reset_index().iloc[:, :4]                                                                                                                    
    tmp = tmp[tmp.code.apply(lambda x: len(str(x)) > 2)]
    t = tmp.groupby(['code', 'subject', 'ed_level']).count().reset_index()
    # оставили все темы, которые встречались хотя бы у двух пользователей
    tmp = pd.merge(tmp, t.loc[t.action_user_id > 1, ['code', 'subject', 'ed_level']], 
               how = 'inner', on = ['code', 'subject', 'ed_level']) 
    tmp['theme'] = list(map(lambda x, y, z: x + '_' + y + '_' + z, tmp.code, tmp.subject, tmp.ed_level))
    tmp.drop(['code', 'subject', 'ed_level'], axis = 1, inplace = True)
    mc['theme'] = list(map(lambda x, y, z: str(x) + '_' + y + '_' + z, mc.code, mc.subject, mc.ed_level))
    return tmp

tmp = make_recom_by_actions(rec_week_num, rec_week_year)

def make_ready_for_recom(rec_week_num, rec_week_year):
    week_delta = 8
    # Создаем даатфрейм с темами на требуемой неделе
    next_themes = lp.loc[(lp.week_num == rec_week_num) & (lp.code.apply(lambda x: len(str(x)) > 2)), 
                         ['teacher_id', 'theme']]
    
    t1 = lp.loc[(lp.week_num < rec_week_num) & (lp.week_num >= rec_week_num - week_delta) & (
        lp.year_num == rec_week_year) & (lp.teacher_id.isin(next_themes.teacher_id.unique())) & (
        lp.code.apply(lambda x: len(str(x)) > 2)), 
                :].groupby(['teacher_id', 'theme']).week_num.count().reset_index().iloc[:, :2]
    t1 = pd.concat([tmp.rename(columns = {'action_user_id': 'teacher_id'}, index = str), t1])
    t1['qty'] = 1
    t1 = t1.groupby(['teacher_id', 'theme']).count().reset_index().iloc[:, :2]
    
    # формируем спарс матрицы
    user_ref = pd.DataFrame({'teacher_id': t1.teacher_id.unique(), 
                             'user_ind': range(t1.teacher_id.unique().shape[0])})
    theme_ref = pd.DataFrame({'theme': t1.theme.unique(), 
                              'theme_ind': range(t1.theme.unique().shape[0])})
    t1 = pd.merge(t1, user_ref, how = 'inner', on = 'teacher_id')
    t1 = pd.merge(t1, theme_ref, how = 'inner', on = 'theme')
    t1['qty'] = 1
    
    size = t1.shape[0]
    row = np.array(t1.user_ind[:size])
    col = np.array(t1.theme_ind[:size])
    data = np.array(t1.qty[:size])
    X = sparse.csr_matrix((data, (row, col))).asfptype()
    
    # Применяем сингулярное разложение
    (U, S, V) = svds(X, k = 100)
    
    # Применяем метод ближайших соседей(KDtree)
    tree = spatial.KDTree(U)
    ready = np.array([next_themes, t1, user_ref, theme_ref, tree, U])
    return ready

next_themes = make_ready_for_recom(rec_week_num, rec_week_year)[0]
t1 = make_ready_for_recom(rec_week_num, rec_week_year)[1]
user_ref = make_ready_for_recom(rec_week_num, rec_week_year)[2]
theme_ref = make_ready_for_recom(rec_week_num, rec_week_year)[3]
tree = make_ready_for_recom(rec_week_num, rec_week_year)[4]
U = make_ready_for_recom(rec_week_num, rec_week_year)[5]

def recomm_by_les_and_act(teacher_id):
    nn_user_ind = tree.query(U[user_ref.loc[user_ref.teacher_id == teacher_id, 'user_ind'].values[0], :], k = 100)[1]
    
    mat_list = mc.loc[mc.theme.isin(next_themes[next_themes.teacher_id.isin(
        user_ref.loc[user_ref.user_ind.isin(nn_user_ind), 
                     'teacher_id'].values)].theme.unique()), :
                     ].groupby(['material_type', 'material_id']).count().reset_index().iloc[:, :2]
    rec_final = pd.merge(marks, mat_list, how = 'inner', on = ['material_type', 'material_id']
                        ).sort_values(by = ['avg_rating', 'marks_qty'], ascending = False).head(10)
    return rec_final

def make_recom_by_les_plan(rec_week_num, rec_week_year, teacher_id):
    # тут отбираем темы, интересные для пользователя на неделе
    tmp = lp.loc[(lp.week_num == rec_week_num) & (lp.year_num == rec_week_year), 
                 ['teacher_id', 'code', 'cis', 'week_num', 'subject', 'ed_level']
                ].groupby(['teacher_id', 'code', 'cis', 'subject', 'ed_level']).count().reset_index().iloc[:, :5]
    tmp = tmp[tmp.code.apply(lambda x: len(str(x)) > 2)]
    # отбираем для пользователя наиболее релевантные материалы по поурочным планам
    rec = pd.merge(pd.merge(tmp.loc[tmp.teacher_id == teacher_id, ['teacher_id', 'code', 'ed_level', 'subject']], 
                            mc, how = 'inner', on = ['code', 'ed_level', 'subject']
                           ).groupby(['material_type', 'material_id']).user_id.count().reset_index().iloc[:, :2], 
                   marks, how = 'inner', on = ['material_type', 'material_id'])
    return rec.sort_values(by = ['avg_rating', 'marks_qty'], ascending = False).head(10)

def fin_rec(teacher_id):
    try:
        res = make_recom_by_les_plan(rec_week_num, rec_week_year, teacher_id)
    except:
        res = recomm_by_les_and_act(teacher_id)
    if res.shape[0] == 0:
        res = recomm_by_les_and_act(teacher_id)
    return res

In [30]:
%%time
res1 = fin_rec(49)

CPU times: user 884 ms, sys: 264 ms, total: 1.15 s
Wall time: 1.14 s


In [31]:
%%time 
res2 = fin_rec(229275)

CPU times: user 1.35 s, sys: 52 ms, total: 1.4 s
Wall time: 1.4 s


In [32]:
res1

,material_type,material_id,avg_rating,marks_qty
83,AtomicObject,1131825,5.0,46
78,AtomicObject,1131804,5.0,45
80,AtomicObject,1131810,5.0,43
89,AtomicObject,1131851,5.0,43
79,AtomicObject,1131807,5.0,42
85,AtomicObject,1131830,5.0,42
20,AtomicObject,430836,5.0,41
81,AtomicObject,1131811,5.0,41
84,AtomicObject,1131828,5.0,41
88,AtomicObject,1131848,5.0,41


In [33]:
res2

,material_id,material_type,avg_rating,marks_qty
2529,29214,AtomicObject,5.0,260
2546,29478,AtomicObject,5.0,260
5196,370870,AtomicObject,5.0,211
5169,368506,AtomicObject,5.0,165
5530,422710,AtomicObject,5.0,162
5579,429376,AtomicObject,5.0,118
5556,426716,AtomicObject,5.0,115
5557,426724,AtomicObject,5.0,115
5559,426779,AtomicObject,5.0,115
5578,429132,AtomicObject,5.0,115


In [ ]:
lp[lp.week_num == 43].head(2)

In [ ]:
lp[lp.teacher_id == 4145839].head(2)

In [44]:
tmp.head(2)

,action_user_id,theme
0,29.0,2.3_Литературное чтение_НОО
1,79.0,2.3_Литературное чтение_НОО


In [ ]:
tmp[tmp.action_user_id == 4145839].head(2)

In [34]:
lp['theme'] = list(map(lambda x, y, z: str(x) + '_' + y + '_' + z, lp.code, lp.subject, lp.ed_level))

In [35]:
lp.head(2)

,lesson_id,teacher_id,date,subject,ed_level,code,cis,week_num,year_num,theme
0,74934616,4145840,2018-09-04,Изобразительное искусство,НОО,1.2,Рисунок,36,2018,1.2_Изобразительное искусство_НОО
1,74934661,4145840,2018-09-11,Изобразительное искусство,НОО,1.1,Восприятие произведений искусства,37,2018,1.1_Изобразительное искусство_НОО


In [36]:
next_themes = lp.loc[(lp.week_num == rec_week_num) & (lp.code.apply(lambda x: len(str(x)) > 2)), 
                     ['teacher_id', 'theme']]

In [37]:
next_themes.head(2)

,teacher_id,theme
38,4145839,3.2.1_Изобразительное искусство_НОО
39,4145839,3.2.2_Изобразительное искусство_НОО


In [38]:
next_themes.teacher_id.unique().shape

(8508,)

In [41]:
t1 = lp.loc[(lp.week_num < rec_week_num) & (lp.week_num >= rec_week_num - week_delta) & (
    lp.year_num == rec_week_year) & (lp.teacher_id.isin(next_themes.teacher_id.unique())) & (
    lp.code.apply(lambda x: len(str(x)) > 2)), 
            :].groupby(['teacher_id', 'theme']).week_num.count().reset_index().iloc[:, :2]
t1 = pd.concat([tmp.rename(columns = {'action_user_id': 'teacher_id'}, index = str), t1])
t1['qty'] = 1
t1 = t1.groupby(['teacher_id', 'theme']).count().reset_index().iloc[:, :2]
print(t1.shape)
t1.head(2)

(1692421, 2)


,teacher_id,theme
0,29.0,2.3_Литературное чтение_НОО
1,29.0,2.7_Литературное чтение_НОО


In [45]:
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
sparse.csr_matrix((data, (row, col))).toarray()#, shape=(3, 3))

array([[1, 0, 2],
       [0, 0, 3],
       [4, 5, 6]], dtype=int64)

In [46]:
user_ref = pd.DataFrame({'teacher_id': t1.teacher_id.unique(), 
                         'user_ind': range(t1.teacher_id.unique().shape[0])})
theme_ref = pd.DataFrame({'theme': t1.theme.unique(), 
                         'theme_ind': range(t1.theme.unique().shape[0])})
t1 = pd.merge(t1, user_ref, how = 'inner', on = 'teacher_id')
t1 = pd.merge(t1, theme_ref, how = 'inner', on = 'theme')
t1['qty'] = 1

In [ ]:
# svd
# def (rec_week_num, rec_week_year, week_delta)

# Задание:
## Реализовать подход для юзеров, у  которых нет ни поурочных планов, ни действий.

In [47]:
t1.head(2)

,teacher_id,theme,user_ind,theme_ind,qty
0,29.0,2.3_Литературное чтение_НОО,0,0,1
1,79.0,2.3_Литературное чтение_НОО,15,0,1


In [48]:
t1.shape

(1692421, 5)

In [49]:
size = t1.shape[0]
row = np.array(t1.user_ind[:size])
col = np.array(t1.theme_ind[:size])
data = np.array(t1.qty[:size])
X = sparse.csr_matrix((data, (row, col))).asfptype()

In [50]:
X[:10, :10]

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [51]:
X.shape

(54358, 14213)

In [52]:
%%time
(U, S, V) = svds(X, k = 100)

CPU times: user 6.15 s, sys: 140 ms, total: 6.29 s
Wall time: 3.43 s


In [53]:
U.shape #пользователи - учителя

(54358, 100)

In [54]:
S.shape

(100,)

In [55]:
V.shape #темы

(100, 14213)

In [56]:
U[:10, :20]

array([[-9.94289468e-04,  6.26527816e-04,  1.19798091e-03,
        -3.21824281e-03,  5.23554446e-04,  5.43493376e-04,
        -6.60143591e-04,  1.24544436e-04,  4.53701436e-04,
         1.48304038e-05, -1.11479280e-04, -3.25078321e-05,
        -4.89793950e-04,  5.75519106e-04,  2.51435813e-04,
        -4.42601320e-04,  1.47416705e-04,  5.43140508e-04,
        -6.38537899e-04,  8.81190454e-04],
       [ 3.23000770e-03, -3.37645860e-04, -2.67890756e-03,
         4.23732906e-04, -7.43207203e-03, -1.71035626e-03,
        -6.86748391e-03,  3.01306027e-03, -1.29955716e-03,
         8.30184098e-03, -6.61778793e-03, -2.48207423e-03,
        -3.37331333e-03, -7.55775594e-03, -1.84727417e-02,
        -1.59656739e-02, -8.80057492e-04,  3.96546230e-04,
        -5.62896240e-03,  4.60255552e-03],
       [-2.31762829e-03, -1.49480335e-03,  2.96886297e-03,
         2.04545000e-03,  4.23623541e-03, -4.87557005e-03,
         4.05432614e-03, -9.24348873e-04, -1.74466137e-03,
        -7.13476758e-03,  3.0

In [57]:
%%time
tree = spatial.KDTree(U)

CPU times: user 2.52 s, sys: 2.05 s, total: 4.57 s
Wall time: 4.57 s


In [58]:
# тут начинается функция
user_ref.loc[user_ref.teacher_id == teacher_id, :]

,teacher_id,user_ind
5,49.0,5


In [59]:
U[user_ref.loc[user_ref.teacher_id == teacher_id, 'user_ind'].values[0], :]

array([-3.52609874e-03, -7.85276761e-04,  1.14245680e-03, -2.74954819e-03,
        1.90605469e-03, -2.28651733e-03, -2.71922047e-03, -2.36862021e-03,
        3.18438697e-03, -8.10477872e-03, -4.17337621e-03, -7.85404096e-03,
       -1.40569848e-02, -1.01748958e-02,  1.11617865e-02,  1.18290623e-02,
       -3.81928148e-04, -9.41944143e-03, -3.88720724e-03, -2.57716870e-03,
        1.03582281e-02,  1.05591094e-03, -2.93360238e-04,  8.62025419e-04,
       -2.12751967e-03,  8.97408904e-04, -8.33131979e-03,  4.32798166e-03,
        3.08573725e-03,  1.01466474e-03, -9.33954927e-04,  1.55702545e-03,
       -1.84458659e-03, -7.92260986e-03, -1.85940213e-02, -2.40840322e-02,
        8.36780871e-03, -5.80270205e-03,  6.38466773e-04, -1.54879548e-03,
        1.59379293e-03, -4.19390170e-03, -7.05520144e-03, -2.10686695e-03,
       -2.78072231e-03, -8.59884445e-03,  1.83716910e-02, -2.35344951e-03,
       -4.98693598e-03, -1.01456419e-02, -9.07919735e-03, -6.79170097e-03,
       -2.38377816e-04, -

In [61]:
user_ref.loc[user_ref.teacher_id == teacher_id, 'user_ind'].values[0]

5

In [62]:
%%time
nn_user_ind = tree.query(U[user_ref.loc[user_ref.teacher_id == teacher_id, 'user_ind'].values[0], :], k = 100)[1]
print(nn_user_ind)

[    5  1576 44781 50408 28062 51266 24273  7282 48821 11079  5145 26806
 51643 16457 10571   146  8787 26834 27125 53483 10352 26445 30504  8672
 53411 44862 25317  1568  7685  8365  3902  7128  8622 42652  5106 13675
 26143  4594 10958   788 35904 27297  5456 53592 24751 22315 14205 53430
 35790 11984 18066 31021  7234 50395 18988 12576  8952 19208 18348  2359
  2712 26189 40352 27206 25015 26388  3302  5744 27710 27012 11074 50625
 28049 53058 30695 24152 50510 22638  8419 13132 42668 40059 50931  9777
  5820  9983 22958  4300 22672 27764 35379 23137 17017  7713  1994   131
 27156 23493 27191 28410]
CPU times: user 712 ms, sys: 0 ns, total: 712 ms
Wall time: 761 ms


In [63]:
user_ref.loc[user_ref.user_ind.isin(nn_user_ind), 'teacher_id'].values

array([4.9000000e+01, 1.7757000e+04, 1.7853000e+04, 7.9029000e+04,
       2.5133400e+05, 2.5232700e+05, 3.1243000e+05, 3.7042300e+05,
       4.5810100e+05, 5.1596700e+05, 5.5927200e+05, 5.9073900e+05,
       6.0892300e+05, 6.5811500e+05, 6.6189900e+05, 2.2851610e+06,
       2.3095740e+06, 2.3198170e+06, 2.4551950e+06, 2.4645100e+06,
       2.4686820e+06, 2.5066410e+06, 2.5091000e+06, 2.5676260e+06,
       2.5736740e+06, 2.5945220e+06, 2.5986720e+06, 2.6072230e+06,
       2.6217060e+06, 2.6881510e+06, 2.6888410e+06, 2.7094990e+06,
       2.7590680e+06, 2.7933430e+06, 2.7985220e+06, 2.7985830e+06,
       3.3187030e+06, 3.5093740e+06, 3.6632660e+06, 3.6662280e+06,
       3.6705890e+06, 3.8315280e+06, 4.0050720e+06, 4.1446350e+06,
       4.1522570e+06, 4.1693190e+06, 4.1780490e+06, 4.2788400e+06,
       4.2854700e+06, 4.2862190e+06, 4.2917830e+06, 4.2945180e+06,
       4.2964110e+06, 4.3034040e+06, 4.3053530e+06, 4.3152700e+06,
       4.3204320e+06, 4.3271190e+06, 4.3731880e+06, 4.3774170e

In [64]:
next_themes[next_themes.teacher_id.isin(user_ref.loc[user_ref.user_ind.isin(nn_user_ind), 
                                                     'teacher_id'].values)].teacher_id.unique()

array([12897539,  4457461,  4398014,  2621706,       49,   252327,
        2598672,  2464510,  4463706,  2709499,  4461223,  4286219,
        4460459,  2594522, 12679666,    17853, 13173449,  4442718,
        2798522,  2798583,  4459920, 13800365,  3509374,  4445796,
       11560484, 12679728,   251334,   559272,   370423,  4402581,
        2759068, 13765278,  4373188, 13957877,  4294518,   658115,
        4461334,  4505850,  4305353,  4497971,  4377417,  2793343,
         608923,    79029, 13871487, 12798273,  2468682,   661899,
       11540452])

In [65]:
next_themes[next_themes.teacher_id.isin(user_ref.loc[user_ref.user_ind.isin(nn_user_ind), 
                                                     'teacher_id'].values)]

,teacher_id,theme
96894,12897539,4.2.8_Русский язык_ООО
96895,12897539,4.3_Русский язык_ООО
96896,12897539,4.2_Русский язык_ООО
96897,12897539,4.3_Русский язык_ООО
96898,12897539,4.5.1_Русский язык_ООО
96899,12897539,4.5.4.1_Русский язык_ООО
96900,12897539,4.2.8_Русский язык_ООО
96901,12897539,4.3_Русский язык_ООО
96902,12897539,4.2_Русский язык_ООО
96903,12897539,4.3_Русский язык_ООО


In [66]:
next_themes[next_themes.teacher_id.isin(user_ref.loc[user_ref.user_ind.isin(nn_user_ind), 
                                                     'teacher_id'].values)].theme.unique()

array(['4.2.8_Русский язык_ООО', '4.3_Русский язык_ООО',
       '4.2_Русский язык_ООО', '4.5.1_Русский язык_ООО',
       '4.5.4.1_Русский язык_ООО', '4.6_Русский язык_ООО',
       '4.6.11_Русский язык_ООО', '4.6.1_Русский язык_ООО',
       '4.6.3_Русский язык_ООО', '4.6.4_Русский язык_ООО',
       '4.6.7_Русский язык_ООО', '4.6.2_Русский язык_ООО',
       '4.2.1_Литература_ООО', '4.2.3_Литература_ООО',
       '4.3.1_Литература_ООО', '4.3.7_Литература_ООО',
       '4.7.4_Русский язык_ООО', '4.7.4.2_Русский язык_ООО',
       '4.7.5_Русский язык_ООО', '4.7.5.1_Русский язык_ООО',
       '4.7.5.2_Русский язык_ООО', '4.7.5.3_Русский язык_ООО',
       '4.7.5.4_Русский язык_ООО', '4.7.5.9_Русский язык_ООО',
       '4.7.7_Русский язык_ООО', '4.7.9_Русский язык_ООО',
       '4.7.9.4_Русский язык_ООО', '4.7.9.2_Русский язык_ООО',
       '4.7.9.3_Русский язык_ООО', '4.7.9.5_Русский язык_ООО',
       '4.1.5_Русский язык_ООО', '4.1.6.1_Русский язык_ООО',
       '4.1.6.2_Русский язык_ООО', '4.1.6.3_Р

In [67]:
mat_list = mc.loc[mc.theme.isin(next_themes[next_themes.teacher_id.isin(
    user_ref.loc[user_ref.user_ind.isin(nn_user_ind), 
                 'teacher_id'].values)].theme.unique()), :
                 ].groupby(['material_type', 'material_id']).count().reset_index().iloc[:, :2]

In [108]:
mat_list

,material_type,material_id
0,AtomicObject,1048
1,AtomicObject,1050
2,AtomicObject,1080
3,AtomicObject,1085
4,AtomicObject,1109
5,AtomicObject,1118
6,AtomicObject,1139
7,AtomicObject,1140
8,AtomicObject,1147
9,AtomicObject,1207


In [109]:
rec_final = pd.merge(marks, mat_list, how = 'inner', on = ['material_type', 'material_id']
        ).sort_values(by = ['avg_rating', 'marks_qty'], ascending = False).head(10)

In [110]:
rec_final

,material_id,material_type,avg_rating,marks_qty
3317,370870,AtomicObject,5.0,211
2958,262762,AtomicObject,5.0,200
4095,465302,AtomicObject,5.0,173
4096,465309,AtomicObject,5.0,173
3369,380426,AtomicObject,5.0,163
4401,522277,AtomicObject,5.0,139
4428,544532,AtomicObject,5.0,116
3694,426724,AtomicObject,5.0,115
3695,426779,AtomicObject,5.0,115
3713,429132,AtomicObject,5.0,115


In [114]:
def make_ready_for_recom(rec_week_num, rec_week_year):
    week_delta = 8
    # Создаем даатфрейм с темами на требуемой неделе
    next_themes = lp.loc[(lp.week_num == rec_week_num) & (lp.code.apply(lambda x: len(str(x)) > 2)), 
                         ['teacher_id', 'theme']]
    
    t1 = lp.loc[(lp.week_num < rec_week_num) & (lp.week_num >= rec_week_num - week_delta) & (
        lp.year_num == rec_week_year) & (lp.teacher_id.isin(next_themes.teacher_id.unique())) & (
        lp.code.apply(lambda x: len(str(x)) > 2)), 
                :].groupby(['teacher_id', 'theme']).week_num.count().reset_index().iloc[:, :2]
    t1 = pd.concat([tmp.rename(columns = {'action_user_id': 'teacher_id'}, index = str), t1])
    t1['qty'] = 1
    t1 = t1.groupby(['teacher_id', 'theme']).count().reset_index().iloc[:, :2]
    
    # формируем спарс матрицы
    user_ref = pd.DataFrame({'teacher_id': t1.teacher_id.unique(), 
                             'user_ind': range(t1.teacher_id.unique().shape[0])})
    theme_ref = pd.DataFrame({'theme': t1.theme.unique(), 
                              'theme_ind': range(t1.theme.unique().shape[0])})
    t1 = pd.merge(t1, user_ref, how = 'inner', on = 'teacher_id')
    t1 = pd.merge(t1, theme_ref, how = 'inner', on = 'theme')
    t1['qty'] = 1
    
    size = t1.shape[0]
    row = np.array(t1.user_ind[:size])
    col = np.array(t1.theme_ind[:size])
    data = np.array(t1.qty[:size])
    X = sparse.csr_matrix((data, (row, col))).asfptype()
    
    # Применяем сингулярное разложение
    (U, S, V) = svds(X, k = 100)
    
    # Применяем метод ближайших соседей(KDtree)
    tree = spatial.KDTree(U)
    ready = np.array([next_themes, t1, user_ref, theme_ref, tree, U])
    return ready

next_themes = make_ready_for_recom(rec_week_num, rec_week_year)[0]
t1 = make_ready_for_recom(rec_week_num, rec_week_year)[1]
user_ref = make_ready_for_recom(rec_week_num, rec_week_year)[2]
theme_ref = make_ready_for_recom(rec_week_num, rec_week_year)[3]
tree = make_ready_for_recom(rec_week_num, rec_week_year)[4]
U = make_ready_for_recom(rec_week_num, rec_week_year)[5]

def recomm_by_les_and_act(teacher_id):
    nn_user_ind = tree.query(U[user_ref.loc[user_ref.teacher_id == teacher_id, 'user_ind'].values[0], :], k = 100)[1]
    
    mat_list = mc.loc[mc.theme.isin(next_themes[next_themes.teacher_id.isin(
        user_ref.loc[user_ref.user_ind.isin(nn_user_ind), 
                     'teacher_id'].values)].theme.unique()), :
                     ].groupby(['material_type', 'material_id']).count().reset_index().iloc[:, :2]
    rec_final = pd.merge(marks, mat_list, how = 'inner', on = ['material_type', 'material_id']
                        ).sort_values(by = ['avg_rating', 'marks_qty'], ascending = False).head(10)
    return rec_final

In [115]:
%%time
recomm_by_les_and_act(49)

CPU times: user 972 ms, sys: 4 ms, total: 976 ms
Wall time: 971 ms


,material_id,material_type,avg_rating,marks_qty
2135,262762,AtomicObject,5.0,200
3037,465282,AtomicObject,5.0,174
3039,465302,AtomicObject,5.0,173
3040,465309,AtomicObject,5.0,173
3038,465288,AtomicObject,5.0,171
2445,380426,AtomicObject,5.0,163
3253,522277,AtomicObject,5.0,139
3275,544532,AtomicObject,5.0,116
2688,424213,AtomicObject,5.0,45
4700,1126648,AtomicObject,5.0,43
